In [3]:
import pytest
import hashlib
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

# Read model (resveratrol from glucose, S. cervisae)
model = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model.reactions.GROWTH.lower_bound=0.04

model.reactions.GROWTH



Reaction identifier,GROWTH
Name,growth
Memory address,0x163abc9a370
Stoichiometry,biomass_c --> biomass -->
GPR,
Lower bound,0.04
Upper bound,1000.0


In [2]:
medium=model.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model.medium = medium
medium

{'EX_nh4_e': 1000.0,
 'EX_glc__D_e': 30,
 'EX_h_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_o2_e': 4,
 'EX_pi_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0}

In [3]:
model.optimize("VVVST1")

,fluxes,reduced_costs
D_LACDcm,0.000000,-1.144917e-16
D_LACDm,0.000000,-1.144917e-16
BTDD_RR,0.000000,0.000000e+00
L_LACD2cm,0.000000,0.000000e+00
13BETGLUSYN,0.527174,1.040834e-17
...,...,...
r_4710,0.000000,0.000000e+00
HA-TAL,0.000000,-1.326619e-01
AT-4CL1,0.000000,0.000000e+00
VVVST1,0.000000,-2.097909e-01


## Lets find in the main model the reactions that we will knock-out

In [4]:

#-	Manual_ko_1: From chorismate to anthranilate (4.1.3.27) 
model.reactions.ANS

Reaction identifier,ANS
Name,anthranilate synthase
Memory address,0x1e2c3e67e20
Stoichiometry,chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c chorismate + L-glutamine --> anthranilate + L-glutamate + H+ + pyruvate
GPR,YER090W and YKL211C
Lower bound,0.0
Upper bound,1000.0


In [5]:
# Manual_ko_2:  From prephenate to phenyl pyruvate (4. 2 . 1. 51) 
model.reactions.PPNDH

Reaction identifier,PPNDH
Name,prephenate dehydratase
Memory address,0x1e2c4319640
Stoichiometry,h_c + pphn_c --> co2_c + h2o_c + phpyr_c H+ + prephenate --> carbon dioxide + H2O + keto-phenylpyruvate
GPR,YNL316C
Lower bound,0.0
Upper bound,1000.0


In [6]:
# Manual_ko_3: 1.1.1.1 acetaldehyde to ethanol Reaction ID "id":":" ALCD2ir"
model.reactions.ALCD2ir

Reaction identifier,ALCD2ir
Name,"alcohol dehydrogenase, (acetaldehyde to ethanol)"
Memory address,0x1e2c47a3c40
Stoichiometry,acald_c + h_c + nadh_c --> etoh_c + nad_c acetaldehyde + H+ + NADH --> ethanol + NAD
GPR,YBR145W or YOL086C
Lower bound,0.0
Upper bound,1000.0


In [7]:
# Manual_ko_4: 1.1.1.2	acetaldehyde to ethanol 2 Reaction ID "id":"ALCD2x_copy2"
model.reactions.ALCD2x

Reaction identifier,ALCD2x
Name,alcohol dehydrogenase (ethanol to acetaldehyde)
Memory address,0x1e2c3e0eb20
Stoichiometry,etoh_c + nad_c --> acald_c + h_c + nadh_c ethanol + NAD --> acetaldehyde + H+ + NADH
GPR,YMR303C
Lower bound,0.0
Upper bound,1000.0


In [8]:
# Manual_ko_5:  Fructose 1,6-DP to fructose 6P (3.1.3.11) "id":"FBP"

model.reactions.FBP

Reaction identifier,FBP
Name,fructose-bisphosphatase
Memory address,0x1e2c3fef640
Stoichiometry,"fdp_c + h2o_c --> f6p_c + pi_c D-fructose 1,6-bisphosphate + H2O --> D-fructose 6-phosphate + phosphate"
GPR,YLR377C
Lower bound,0.0
Upper bound,1000.0





## First Model (Manual_ko_1)
### From chorismate to anthranilate (Enzyme: 4.1.3.27) 

In [9]:
# Read model (resveratrol from glucose, S. cervisae)
model_Manual_ko_1 = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model_Manual_ko_1.reactions.GROWTH.lower_bound=0.04
medium=model_Manual_ko_1.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model_Manual_ko_1.medium = medium


In [10]:
#Knock out changes the boundaries to zero
model_Manual_ko_1.reactions.ANS.knock_out()

In [11]:
model_Manual_ko_1.reactions.ANS

Reaction identifier,ANS
Name,anthranilate synthase
Memory address,0x1e2c6edebe0
Stoichiometry,chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c chorismate + L-glutamine --> anthranilate + L-glutamate + H+ + pyruvate
GPR,YER090W and YKL211C
Lower bound,0
Upper bound,0


In [12]:
# Yield and productivity from section 3
model.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x1e2c58bba90
Stoichiometry,e4_coumaroyl_coa + 3.0 h_c + 3.0 malcoa_c --> 4.0 co2_c + 4.0 coa_c + trans_resv + 3.0 H+ + 3.0 malonyl-CoA --> 4.0 carbon dioxide + 4.0 coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
with model_Manual_ko_1:
    model_Manual_ko_1.reactions.ANS.knock_out()
    model_Manual_ko_1.objective = model_Manual_ko_1.reactions.VVVST1
    
    resveratrol_max_production = model_Manual_ko_1.optimize().objective_value
    print("Maximum resveratrol productivity = ",resveratrol_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model_Manual_ko_1.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = resveratrol_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")
    
    # Calculation in C-mol (TBD)
#resv_max_prod_cmol = (resveratrol_max_production)/(resveatrol)
#resv_max_yield_cmol = (max_yield*resveatrol)/(glucose)
#print('Maximum productivity of lycopene =', resv_max_prod_cmol, 'cmol-resv/gDW*h')
#print('Maximum theoretical yield =', resv_max_yield_cmol, 'cmol-resv/cmol-glucose')


Maximum resveratrol productivity =  0.0 mmol/gDW*h
Maximum theoretical yield is =  0.0 mmol/gDW*h


C:\Users\ivaro\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\ivaro\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)





## Second Model (Manual_ko_2)
### From prephenate to phenyl pyruvate (Enzyme 4. 2 . 1. 51) 

In [14]:
# Read model (resveratrol from glucose, S. cervisae)
model_Manual_ko_2 = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model_Manual_ko_2.reactions.GROWTH.lower_bound=0.04
medium=model_Manual_ko_2.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model_Manual_ko_2.medium = medium

In [15]:
#Knock out changes the boundaries to zero
model_Manual_ko_2.reactions.PPNDH.knock_out()
model_Manual_ko_2.reactions.PPNDH

Reaction identifier,PPNDH
Name,prephenate dehydratase
Memory address,0x1e2c9deeeb0
Stoichiometry,h_c + pphn_c --> co2_c + h2o_c + phpyr_c H+ + prephenate --> carbon dioxide + H2O + keto-phenylpyruvate
GPR,YNL316C
Lower bound,0
Upper bound,0


In [16]:
with model_Manual_ko_2:
    model_Manual_ko_2.objective = model_Manual_ko_2.reactions.VVVST1
    model_Manual_ko_2.reactions.PPNDH.knock_out()
    
    #also knocking out glucose exchange reaction just to see if there is effect
    #model_Manual_ko_2.reactions.GLCt1.knock_out()
    resveratrol_max_production = model_Manual_ko_2.optimize().objective_value
    print("Maximum resveratrol productivity = ",resveratrol_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model_Manual_ko_2.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = resveratrol_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")
model_Manual_ko_2.reactions.PPNDH

Maximum resveratrol productivity =  1.799119497708899e-17 mmol/gDW*h
Maximum theoretical yield is =  1.1648013662309603e-16 mmol/gDW*h


C:\Users\ivaro\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\ivaro\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Reaction identifier,PPNDH
Name,prephenate dehydratase
Memory address,0x1e2c9deeeb0
Stoichiometry,h_c + pphn_c --> co2_c + h2o_c + phpyr_c H+ + prephenate --> carbon dioxide + H2O + keto-phenylpyruvate
GPR,YNL316C
Lower bound,0
Upper bound,0


## Third Model (Manual_ko_3)¶
### (Enz. 1.1.1.1) acetaldehyde to ethanol Reaction ID "id":":" ALCD2ir"

In [17]:
# Read model (resveratrol from glucose, S. cervisae)
model_Manual_ko_3 = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model_Manual_ko_3.reactions.GROWTH.lower_bound=0.04
medium=model_Manual_ko_3.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model_Manual_ko_3.medium = medium

In [18]:
with model_Manual_ko_3:
    model_Manual_ko_3.objective = model_Manual_ko_3.reactions.VVVST1
    model_Manual_ko_3.reactions.ALCD2ir.knock_out()
    
    #also knocking out glucose exchange reaction just to see if there is effect
    #model_Manual_ko_2.reactions.GLCt1.knock_out()
    resveratrol_max_production = model_Manual_ko_3.optimize().objective_value
    print("Maximum resveratrol productivity = ",resveratrol_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model_Manual_ko_3.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = resveratrol_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")
model_Manual_ko_3.reactions.ALCD2ir

Maximum resveratrol productivity =  3.3174079489656467 mmol/gDW*h
Maximum theoretical yield is =  0.11058026496552156 mmol/gDW*h


Reaction identifier,ALCD2ir
Name,"alcohol dehydrogenase, (acetaldehyde to ethanol)"
Memory address,0x1e2cbcbffd0
Stoichiometry,acald_c + h_c + nadh_c --> etoh_c + nad_c acetaldehyde + H+ + NADH --> ethanol + NAD
GPR,YBR145W or YOL086C
Lower bound,0.0
Upper bound,1000.0


## Fourth Model (Manual_ko_4)¶
### (Enz. 1.1.1.2) acetaldehyde to ethanol Reaction ID "id":":" ALCD2ir"


In [19]:
# Read model (resveratrol from glucose, S. cervisae)
model_Manual_ko_4 = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model_Manual_ko_4.reactions.GROWTH.lower_bound=0.04
medium=model_Manual_ko_4.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model_Manual_ko_4.medium = medium

model_Manual_ko_4.reactions.ALCD2x.knock_out()
model_Manual_ko_4.reactions.ALCD2ir.knock_out()

In [20]:
with model_Manual_ko_4:
    model_Manual_ko_4.objective = model_Manual_ko_4.reactions.VVVST1
    model_Manual_ko_4.reactions.ALCD2ir.knock_out()
    #model_Manual_ko_4.reactions.ALCD2ir.knock_out()
    
    resveratrol_max_production = model_Manual_ko_4.optimize().objective_value
    print("Maximum resveratrol productivity = ",resveratrol_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model_Manual_ko_4.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = resveratrol_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")
model_Manual_ko_4.reactions.ALCD2ir

Maximum resveratrol productivity =  3.317407863831894 mmol/gDW*h
Maximum theoretical yield is =  0.1105802621277298 mmol/gDW*h


Reaction identifier,ALCD2ir
Name,"alcohol dehydrogenase, (acetaldehyde to ethanol)"
Memory address,0x1e2ce853c70
Stoichiometry,acald_c + h_c + nadh_c --> etoh_c + nad_c acetaldehyde + H+ + NADH --> ethanol + NAD
GPR,YBR145W or YOL086C
Lower bound,0
Upper bound,0


## Fifth Model (Manual_ko_5)¶
### Fructose 1,6-DP to fructose 6P (3.1.3.11) "id":"FBP"

In [21]:
model_Manual_ko_5 = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model_Manual_ko_5.reactions.GROWTH.lower_bound=0.04
medium=model_Manual_ko_5.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model_Manual_ko_5.medium = medium

model_Manual_ko_5.reactions.FBP.knock_out()


In [22]:
with model_Manual_ko_5:
    model_Manual_ko_5.objective = model_Manual_ko_5.reactions.VVVST1
    model_Manual_ko_5.reactions.FBP.knock_out()
    #model_Manual_ko_4.reactions.ALCD2ir.knock_out()
    
    resveratrol_max_production = model_Manual_ko_5.optimize().objective_value
    print("Maximum resveratrol productivity = ",resveratrol_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model_Manual_ko_5.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = resveratrol_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")
model_Manual_ko_5.reactions.FBP

Maximum resveratrol productivity =  3.317407948965657 mmol/gDW*h
Maximum theoretical yield is =  0.1105802649655219 mmol/gDW*h


Reaction identifier,FBP
Name,fructose-bisphosphatase
Memory address,0x1e2d0f67d30
Stoichiometry,"fdp_c + h2o_c --> f6p_c + pi_c D-fructose 1,6-bisphosphate + H2O --> D-fructose 6-phosphate + phosphate"
GPR,YLR377C
Lower bound,0
Upper bound,0


## Single Deletions (https://cobrapy.readthedocs.io/en/latest/deletions.html)
### By general analysis of de flux using cobrapy

In [10]:
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
model_single_deletions = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model_single_deletions.reactions.GROWTH.bounds=0.04, 1000

medium=model_single_deletions.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model_single_deletions.medium = medium
model_single_deletions.objective  = model_single_deletions.reactions.VVVST1

In [5]:
model_single_deletions

Name,yeastGEM_v8__46__6__46__2
Memory address,0x0163ae1e1610
Number of metabolites,2747
Number of reactions,4067
Number of groups,106
Objective expression,1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [25]:

single_reaction_deletion(model_single_deletions, model_single_deletions.reactions[:50])

,ids,growth,status
0,{PDE4},3.317408,optimal
1,{DRPPPNRED25645},NaN,infeasible
2,{PRMICI},NaN,infeasible
3,{MCITSm},3.317408,optimal
4,{BPNT},3.240766,optimal
5,{IPMD},NaN,infeasible
6,{HACD6p},3.317408,optimal
7,{PDE3},3.317408,optimal
8,{PDE1},3.317408,optimal
9,{BTDD_RR},3.317408,optimal


In [26]:
deletions=single_reaction_deletion(model_single_deletions, model_single_deletions.reactions)
deletions = deletions.sort_values(by='growth', ascending=False)
print(deletions.head(10))


                            ids    growth   status
855                    {GROWTH}  3.509689  optimal
1972               {ATPM_copy2}  3.347494  optimal
1156             {PSYN181161mm}  3.317408  optimal
1154                  {THICOXI}  3.317408  optimal
1153                   {r_4645}  3.317408  optimal
1152  {PPPL11612181316041813mm}  3.317408  optimal
1151                   {GLNt2r}  3.317408  optimal
1150                 {THIORDXi}  3.317408  optimal
1149              {DAG180161tn}  3.317408  optimal
1148                 {3OAR60_m}  3.317408  optimal


In [27]:
max_fluxes_deletion = deletions.growth[deletions.growth == max(deletions.growth)]
max_fluxes_deletion

855    3.509689
Name: growth, dtype: float64

## Double reaction deletions


In [ ]:
deletions2=double_reaction_deletion(model_single_deletions, model_single_deletions.reactions)
deletions2 = deletions2.sort_values(by='growth', ascending=False)
print(deletions2.head(10))

## Single Gene Deletions

In [12]:
deletions3=single_gene_deletion(model_single_deletions, model_single_deletions.genes)
deletions3 = deletions3.sort_values(by='growth', ascending=False)

print(deletions3.head(30))

            ids    growth   status
33    {YPL087W}  3.317408  optimal
16    {YHL011C}  3.317408  optimal
31    {YGL224C}  3.317408  optimal
30    {YNL130C}  3.317408  optimal
29    {YDL052C}  3.317408  optimal
28    {YDL238C}  3.317408  optimal
26    {YER062C}  3.317408  optimal
25    {YNR057C}  3.317408  optimal
23    {YDR272W}  3.317408  optimal
22    {YNL065W}  3.317408  optimal
21    {YCL017C}  3.317408  optimal
20    {YKL212W}  3.317408  optimal
19    {YML042W}  3.317408  optimal
18    {YDR072C}  3.317408  optimal
17    {YOR126C}  3.317408  optimal
15    {YER015W}  3.317408  optimal
34    {YBR018C}  3.317408  optimal
14    {YDL040C}  3.317408  optimal
13    {YLR092W}  3.317408  optimal
35    {YDL182W}  3.317408  optimal
37    {YJL071W}  3.317408  optimal
38  {YLL018C-A}  3.317408  optimal
39    {YNR056C}  3.317408  optimal
40    {YLR348C}  3.317408  optimal
41    {YML082W}  3.317408  optimal
43    {YDR062W}  3.317408  optimal
44    {YDR302W}  3.317408  optimal
45    {YGR254W}  3.3

## Double Gene Deletions

In [ ]:
deletions4=double_gene_deletion(model_single_deletions, model_single_deletions.genes)
deletions4 = deletions4.sort_values(by='growth', ascending=False)
print(deletions4.head(10))

## Other deletions
### ATPM knock out {
"id":"ATPM",
"name":"ATP maintenance requirement",
"metabolites":{
"adp_c":1.0,
"atp_c":-1.0,
"h2o_c":-1.0,
"h_c":1.0,
"pi_c":1.0
}


In [ ]:
model_Manual_ko_6 = read_sbml_model('../data/models/yeast8_resv_glc.xml')
model_Manual_ko_6.reactions.GROWTH.lower_bound=0.04
medium=model_Manual_ko_6.medium
medium['EX_glc__D_e'] = 30
medium['EX_o2_e'] = 4
model_Manual_ko_6.medium = medium

In [ ]:
with model_Manual_ko_6:
    model_Manual_ko_6.objective = model_Manual_ko_6.reactions.VVVST1
    model_Manual_ko_6.reactions.ATPM.knock_out()
  
    
    resveratrol_max_production = model_Manual_ko_6.optimize().objective_value
    print("Maximum resveratrol productivity = ",resveratrol_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model_Manual_ko_6.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = resveratrol_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")